In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os

from dask.distributed import Client

from ase.build import bulk
from abtem import FrozenPhonons, Probe, Potential

os.environ["MKL_NUM_THREADS"] = "1"

Runtime: 1.9 s per frozen phonon on 8-core Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz

In [4]:
client = Client()

In [9]:
silicon = bulk('Si', crystalstructure='diamond', cubic=True)
silicon *= (74,74,2)

In [62]:
frozen_phonons = FrozenPhonons(silicon, 16, {'Si':.1})

potential = Potential(frozen_phonons, 
                      gpts=2048,
                      device='cpu',
                      projection='infinite', 
                      slice_thickness=.5)

wave = Probe(energy=100e3, semiangle_cutoff=9.4, device='cpu')

In [63]:
measurement = wave.multislice(potential, (0,0)).diffraction_patterns().mean(0)

In [64]:
%%time
measurement.compute()

Wall time: 31.3 s
